In [1]:
# Classification Dummy Data and Model Evaluation with PyTorch

# Importing required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools

# Load dataset provided by user (heart.csv) or generate dummy data if needed
def load_data(file_path):
    # Load the dataset from CSV
    data = pd.read_csv(file_path)
    X = data.iloc[:, :-1].values  # Features
    y = data.iloc[:, -1].values   # Labels
    return X, y

# Generate dummy data as fallback
def generate_dummy_data(n_samples=1000, n_features=20, n_classes=2):
    X, y = make_classification(
        n_samples=n_samples, n_features=n_features, n_classes=n_classes,
        n_informative=15, n_redundant=5, random_state=42
    )
    return X, y

# Load heart.csv or fallback to dummy data
try:
    X, y = load_data('heart.csv')
except Exception as e:
    print("Error loading dataset, using dummy data instead:", e)
    X, y = generate_dummy_data()

# Preprocessing data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define a generic Multi-Layer Perceptron (MLP) class
def create_mlp(input_size, hidden_layers, activation_fn):
    layers = []
    current_size = input_size

    # Adding hidden layers dynamically
    for neurons in hidden_layers:
        layers.append(nn.Linear(current_size, neurons))
        layers.append(activation_fn())
        current_size = neurons

    # Output layer (binary classification -> 2 neurons)
    layers.append(nn.Linear(current_size, 2))

    # Return model
    return nn.Sequential(*layers)

# Training loop
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs, batch_size):
    train_loader = torch.utils.data.DataLoader(
        dataset=torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=torch.utils.data.TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False
    )

    train_losses, test_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        train_losses.append(epoch_loss / len(train_loader))

        # Evaluate model after every epoch
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                preds = model(X_batch).argmax(dim=1)
                all_preds.extend(preds.numpy())
                all_labels.extend(y_batch.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        test_accuracies.append(accuracy)

    return train_losses, test_accuracies

# Experimentation setup
def experiment():
    input_size = X_train.shape[1]

    # Hyperparameters to test
    hidden_layer_configs = [[4], [8], [16, 8], [32, 16, 8]]  # Different hidden layer sizes
    activation_fns = [nn.ReLU, nn.Sigmoid, nn.Tanh, nn.Softmax, nn.Identity]  # Activation functions
    epoch_counts = [10, 25, 50]
    learning_rates = [0.1, 0.01, 0.001]
    batch_sizes = [16, 32, 64, 128]

    results = []

    for hl_config, activation_fn, epochs, lr, batch_size in itertools.product(
            hidden_layer_configs, activation_fns, epoch_counts, learning_rates, batch_sizes):

        print(f"Testing: {hl_config}, {activation_fn.__name__}, {epochs} epochs, LR={lr}, Batch={batch_size}")

        # Create model and set up training
        model = create_mlp(input_size, hl_config, activation_fn)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        train_losses, test_accuracies = train_model(
            model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs, batch_size
        )

        # Save results for analysis
        results.append({
            'hidden_layers': hl_config,
            'activation_fn': activation_fn.__name__,
            'epochs': epochs,
            'learning_rate': lr,
            'batch_size': batch_size,
            'final_accuracy': test_accuracies[-1]
        })

    return results

# Run experiments
results = experiment()

# Save results for analysis
results_df = pd.DataFrame(results)
results_df.to_csv('mlp_experiment_results.csv', index=False)

# Summarize and analyze results
print("Experiment completed. Results saved to 'mlp_experiment_results.csv'.")


Testing: [4], ReLU, 10 epochs, LR=0.1, Batch=16
Testing: [4], ReLU, 10 epochs, LR=0.1, Batch=32
Testing: [4], ReLU, 10 epochs, LR=0.1, Batch=64
Testing: [4], ReLU, 10 epochs, LR=0.1, Batch=128
Testing: [4], ReLU, 10 epochs, LR=0.01, Batch=16
Testing: [4], ReLU, 10 epochs, LR=0.01, Batch=32
Testing: [4], ReLU, 10 epochs, LR=0.01, Batch=64
Testing: [4], ReLU, 10 epochs, LR=0.01, Batch=128
Testing: [4], ReLU, 10 epochs, LR=0.001, Batch=16
Testing: [4], ReLU, 10 epochs, LR=0.001, Batch=32
Testing: [4], ReLU, 10 epochs, LR=0.001, Batch=64
Testing: [4], ReLU, 10 epochs, LR=0.001, Batch=128
Testing: [4], ReLU, 25 epochs, LR=0.1, Batch=16
Testing: [4], ReLU, 25 epochs, LR=0.1, Batch=32
Testing: [4], ReLU, 25 epochs, LR=0.1, Batch=64
Testing: [4], ReLU, 25 epochs, LR=0.1, Batch=128
Testing: [4], ReLU, 25 epochs, LR=0.01, Batch=16
Testing: [4], ReLU, 25 epochs, LR=0.01, Batch=32
Testing: [4], ReLU, 25 epochs, LR=0.01, Batch=64
Testing: [4], ReLU, 25 epochs, LR=0.01, Batch=128
Testing: [4], ReLU,

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules

Testing: [4], Softmax, 10 epochs, LR=0.1, Batch=32


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules

Testing: [4], Softmax, 10 epochs, LR=0.1, Batch=64


...

Testing: [32, 16, 8], Softmax, 50 epochs, LR=0.001, Batch=64


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules

Testing: [32, 16, 8], Softmax, 50 epochs, LR=0.001, Batch=128


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules

Testing: [32, 16, 8], Identity, 10 epochs, LR=0.1, Batch=16
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.1, Batch=32
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.1, Batch=64
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.1, Batch=128
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.01, Batch=16
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.01, Batch=32
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.01, Batch=64
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.01, Batch=128
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.001, Batch=16
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.001, Batch=32
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.001, Batch=64
Testing: [32, 16, 8], Identity, 10 epochs, LR=0.001, Batch=128
Testing: [32, 16, 8], Identity, 25 epochs, LR=0.1, Batch=16
Testing: [32, 16, 8], Identity, 25 epochs, LR=0.1, Batch=32
Testing: [32, 16, 8], Identity, 25 epochs, LR=0.1, Batch=64
Testing: [32, 16, 8], Identity, 25 epochs, LR=0.1, Batch=128
Testing: [32, 16, 8], Id